In [16]:
import pandas as pd
import requests
import json
from pprint import pprint
import matplotlib.pyplot as plt
import math

# # https://towardsdatascience.com/how-to-visualize-data-on-top-of-a-map-in-python-using-the-geoviews-library-part-2-e61a48ee6c3d
# import geopandas as gpd
# import geoviews as gv
# import geoviews.tile_sources as gvts
# from geoviews import opts
# gv.extension('bokeh')

# # https://github.com/Toblerity/Fiona
# import fiona
# from fiona import Feature, Geometry
# from shapely.geometry import mapping, shape

# import warnings

# warnings.filterwarnings('ignore')

from api_keys import geoapify_key

In [17]:
world_data_df = pd.read_csv('Datasets/world_population.csv')

base_url = "https://api.geoapify.com/v1/geocode/search"
params = {
    "apiKey":geoapify_key,
    "format":"json",
    "limit": 1
}



world_data_df = world_data_df.reindex(columns = world_data_df.columns.tolist() + ['geo_country'])
world_data_df = world_data_df.reindex(columns = world_data_df.columns.tolist() + ['iso_two'])
# world_data_df = world_data_df.reindex(columns = world_data_df.columns.tolist() + ['iso_three'])

world_data_df.head()

,Rank,CCA3,Country/Territory,Capital,Continent,2022 Population,2020 Population,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage,geo_country,iso_two
0,36,AFG,Afghanistan,Kabul,Asia,41128771,38972230,33753499,28189672,19542982,10694796,12486631,10752971,652230,63.0587,1.0257,0.52,NaN,NaN
1,138,ALB,Albania,Tirana,Europe,2842321,2866849,2882481,2913399,3182021,3295066,2941651,2324731,28748,98.8702,0.9957,0.04,NaN,NaN
2,34,DZA,Algeria,Algiers,Africa,44903225,43451666,39543154,35856344,30774621,25518074,18739378,13795915,2381741,18.8531,1.0164,0.56,NaN,NaN
3,213,ASM,American Samoa,Pago Pago,Oceania,44273,46189,51368,54849,58230,47818,32886,27075,199,222.4774,0.9831,0.00,NaN,NaN
4,203,AND,Andorra,Andorra la Vella,Europe,79824,77700,71746,71519,66097,53569,35611,19860,468,170.5641,1.0100,0.00,NaN,NaN


In [18]:
# https://github.com/stefangabos/world_countries/blob/master/data/countries/_combined/countries.csv
countrynames = pd.read_csv('Datasets/countries.csv')
countrynames

,id,alpha2,alpha3,ar,bg,cs,da,de,el,en,...,ro,ru,sk,sl,sr,sv,th,uk,zh,zh-tw
0,4,af,afg,أفغانستان,Афганистан,Afghánistán,Afghanistan,Afghanistan,Αφγανιστάν,Afghanistan,...,Afganistan,Афганистан,Afganistan,Afganistan,Avganistan,Afghanistan,อัฟกานิสถาน,Афганістан,阿富汗,阿富汗
1,8,al,alb,ألبانيا,Албания,Albánie,Albanien,Albanien,Αλβανία,Albania,...,Albania,Албания,Albánsko,Albanija,Albanija,Albanien,แอลเบเนีย,Албанія,阿尔巴尼亚,阿爾巴尼亞
2,12,dz,dza,الجزائر,Алжир,Alžírsko,Algeriet,Algerien,Αλγερία,Algeria,...,Algeria,Алжир,Alžírsko,Alžirija,Alžir,Algeriet,แอลจีเรีย,Алжир,阿尔及利亚,阿爾及利亞
3,20,ad,and,أندورا,Андора,Andorra,Andorra,Andorra,Ανδόρρα,Andorra,...,Andorra,Андорра,Andorra,Andora,Andora,Andorra,อันดอร์รา,Андорра,安道尔,安道爾
4,24,ao,ago,أنغولا,Ангола,Angola,Angola,Angola,Ανγκόλα,Angola,...,Angola,Ангола,Angola,Angola,Angola,Angola,แองโกลา,Ангола,安哥拉,安哥拉
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,862,ve,ven,فنزويلا,Венецуела,Venezuela,Venezuela,Venezuela,Βενεζουέλα,Venezuela (Bolivarian Republic of),...,Venezuela,Венесуэла,Venezuela,Venezuela,Venecuela,Venezuela,เวเนซุเอลา,Венесуела,委内瑞拉,委內瑞拉
189,704,vn,vnm,فيتنام,Виетнам,Vietnam,Vietnam,Vietnam,Βιετνάμ,Viet Nam,...,Vietnam,Вьетнам,Vietnam,Vietnam,Vijetnam,Vietnam,เวียดนาม,В'єтнам,越南,越南
190,887,ye,yem,اليمن,Йемен,Jemen,Yemen,Jemen,Υεμένη,Yemen,...,Yemen,Йемен,Jemen,Jemen,Jemen,Jemen,เยเมน,Ємен,也门,葉門
191,894,zm,zmb,زامبيا,Замбия,Zambie,Zambia,Sambia,Ζάμπια,Zambia,...,Zambia,Замбия,Zambia,Zambija,Zambija,Zambia,แซมเบีย,Замбія,赞比亚,尚比亞


In [26]:
world_data_df
params['country'] = 'New Caledonia'
current_country = requests.get(base_url, params).json()

pprint(current_country)
world_data_df

{'query': {'parsed': {'country': 'New Caledonia', 'expected_type': 'unknown'},
           'text': ''},
 'results': [{'address_line1': 'France',
              'address_line2': '',
              'archipelago': 'New Caledonia',
              'bbox': {'lat1': -23.2217509,
                       'lat2': -17.6868616,
                       'lon1': 157.9847541,
                       'lon2': 172.3057152},
              'category': 'administrative',
              'country': 'France',
              'country_code': 'fr',
              'datasource': {'attribution': '© OpenStreetMap contributors',
                             'license': 'Open Database License',
                             'sourcename': 'openstreetmap',
                             'url': 'https://www.openstreetmap.org/copyright'},
              'formatted': 'France',
              'lat': -21.3019905,
              'lon': 165.4880773,
              'place_id': '51702d49549eaf644059e333d93f4f4d35c0f00101f9011bff330000000000c0020b',

,Rank,CCA3,Country/Territory,Capital,Continent,2022 Population,2020 Population,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage,geo_country,iso_two,pop_change
0,36,AFG,Afghanistan,Kabul,Asia,41128771,38972230,33753499,28189672,19542982,10694796,12486631,10752971,652230,63.0587,1.0257,0.52,Afghanistan,af,30375800
1,138,ALB,Albania,Tirana,Europe,2842321,2866849,2882481,2913399,3182021,3295066,2941651,2324731,28748,98.8702,0.9957,0.04,Albania,al,517590
2,34,DZA,Algeria,Algiers,Africa,44903225,43451666,39543154,35856344,30774621,25518074,18739378,13795915,2381741,18.8531,1.0164,0.56,Algeria,dz,31107310
4,203,AND,Andorra,Andorra la Vella,Europe,79824,77700,71746,71519,66097,53569,35611,19860,468,170.5641,1.0100,0.00,Andorra,ad,59964
5,42,AGO,Angola,Luanda,Africa,35588987,33428485,28127721,23364185,16394062,11828638,8330047,6029700,1246700,28.5466,1.0315,0.45,Angola,ao,29559287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,16,VNM,Vietnam,Hanoi,Asia,98186856,96648685,92191398,87411012,79001142,66912613,52968270,41928849,331212,296.4472,1.0074,1.23,Vietnam,vn,56258007
230,172,ESH,Western Sahara,El Aaiún,Africa,575986,556048,491824,413296,270375,178529,116775,76371,266000,2.1654,1.0184,0.01,Western Sahara,eh,499615
231,46,YEM,Yemen,Sanaa,Asia,33696614,32284046,28516545,24743946,18628700,13375121,9204938,6843607,527968,63.8232,1.0217,0.42,Yemen,ye,26853007
232,63,ZMB,Zambia,Lusaka,Africa,20017675,18927715,16248230,13792086,9891136,7686401,5720438,4281671,752612,26.5976,1.0280,0.25,Zambia,zm,15736004


In [28]:
for index, row in world_data_df.iterrows():
    params['country'] = world_data_df.loc[index, 'Country/Territory']
    current_country = requests.get(base_url, params).json()
    
    try: 
        world_data_df.loc[index, 'geo_country'] = current_country['results'][0]['country']
        world_data_df.loc[index, 'iso_two'] = current_country['results'][0]['country_code']
        print(f"Indexing: {world_data_df.loc[index, 'geo_country']} as {world_data_df.loc[index, 'iso_two']} and confidence: {current_country['results'][0]['rank']['confidence']}")
#         print(current_country['results'][0]['rank']['confidence']
    except IndexError:
        print(f"Dropping: {current_country}")
        world_data_df = world_data_df.drop(labels=index, axis=0)
        


    


Indexing: Afghanistan as af and confidence: 1
Indexing: Albania as al and confidence: 1
Indexing: Algeria as dz and confidence: 1
Indexing: Andorra as ad and confidence: 1
Indexing: Angola as ao and confidence: 1
Indexing: Anguilla as ai and confidence: 1
Indexing: Antigua and Barbuda as ag and confidence: 1
Indexing: Argentina as ar and confidence: 1
Indexing: Armenia as am and confidence: 1
Indexing: Aruba as aw and confidence: 1
Indexing: Australia as au and confidence: 1
Indexing: Austria as at and confidence: 1
Indexing: Azerbaijan as az and confidence: 1
Indexing: The Bahamas as bs and confidence: 1
Indexing: Bahrain as bh and confidence: 1
Indexing: Bangladesh as bd and confidence: 1
Indexing: Barbados as bb and confidence: 1
Indexing: Belarus as by and confidence: 1
Indexing: Belgium as be and confidence: 1
Indexing: Belize as bz and confidence: 1
Indexing: Benin as bj and confidence: 1
Indexing: Bermuda as bm and confidence: 1
Indexing: Bhutan as bt and confidence: 1
Indexing:

Indexing: Sudan as sd and confidence: 1
Indexing: Suriname as sr and confidence: 1
Indexing: Sweden as se and confidence: 1
Indexing: Switzerland as ch and confidence: 1
Indexing: Syria as sy and confidence: 1
Indexing: Taiwan as tw and confidence: 1
Indexing: Tajikistan as tj and confidence: 1
Indexing: Tanzania as tz and confidence: 1
Indexing: Thailand as th and confidence: 1
Indexing: East Timor as tl and confidence: 1
Indexing: Togo as tg and confidence: 1
Indexing: Tokelau as tk and confidence: 1
Indexing: Tonga as to and confidence: 1
Indexing: Trinidad and Tobago as tt and confidence: 1
Indexing: Tunisia as tn and confidence: 1
Indexing: Turkey as tr and confidence: 1
Indexing: Turkmenistan as tm and confidence: 1
Indexing: Turks and Caicos Islands as tc and confidence: 1
Indexing: Tuvalu as tv and confidence: 1
Indexing: Uganda as ug and confidence: 1
Indexing: Ukraine as ua and confidence: 1
Indexing: United Arab Emirates as ae and confidence: 1
Indexing: United Kingdom as gb

In [20]:
# https://github.com/stefangabos/world_countries/blob/master/data/countries/_combined/countries.csv

countrynames = pd.read_csv('Datasets/countries.csv')
countrynames.rename(columns={'alpha2': 'iso_two'}, inplace=True)
countrynames = countrynames[['iso_two', 'alpha3']]
countrynames
countryiso = pd.merge(world_data_df, countrynames, on='iso_two', how='left')
countryiso['iso_two'].value_counts()


lc    2
sx    2
fr    2
ps    1
nz    1
     ..
gd    1
gt    1
gg    1
gn    1
zw    1
Name: iso_two, Length: 217, dtype: int64

In [21]:
#  / world_data_df['1970 Population'] 41 92
print(world_data_df['geo_country'].value_counts())
world_data_df.loc[world_data_df['geo_country'] == 'Sint Maarten']

world_data_df['pop_change'] = (world_data_df['2022 Population'] - world_data_df['1970 Population'])
world_data_df
# world_data_df = world_data_df.drop(index=(41, 92), axis=0)
# world_data_df.loc[world_data_df['geo_country'] == 'India']

France              2
Sint Maarten        2
Papua New Guinea    1
Niger               1
Nigeria             1
                   ..
Guatemala           1
Guernsey            1
Guinea              1
Guinea-Bissau       1
Zimbabwe            1
Name: geo_country, Length: 218, dtype: int64


,Rank,CCA3,Country/Territory,Capital,Continent,2022 Population,2020 Population,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage,geo_country,iso_two,pop_change
0,36,AFG,Afghanistan,Kabul,Asia,41128771,38972230,33753499,28189672,19542982,10694796,12486631,10752971,652230,63.0587,1.0257,0.52,Afghanistan,af,30375800
1,138,ALB,Albania,Tirana,Europe,2842321,2866849,2882481,2913399,3182021,3295066,2941651,2324731,28748,98.8702,0.9957,0.04,Albania,al,517590
2,34,DZA,Algeria,Algiers,Africa,44903225,43451666,39543154,35856344,30774621,25518074,18739378,13795915,2381741,18.8531,1.0164,0.56,Algeria,dz,31107310
4,203,AND,Andorra,Andorra la Vella,Europe,79824,77700,71746,71519,66097,53569,35611,19860,468,170.5641,1.0100,0.00,Andorra,ad,59964
5,42,AGO,Angola,Luanda,Africa,35588987,33428485,28127721,23364185,16394062,11828638,8330047,6029700,1246700,28.5466,1.0315,0.45,Angola,ao,29559287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,16,VNM,Vietnam,Hanoi,Asia,98186856,96648685,92191398,87411012,79001142,66912613,52968270,41928849,331212,296.4472,1.0074,1.23,Vietnam,vn,56258007
230,172,ESH,Western Sahara,El Aaiún,Africa,575986,556048,491824,413296,270375,178529,116775,76371,266000,2.1654,1.0184,0.01,Western Sahara,eh,499615
231,46,YEM,Yemen,Sanaa,Asia,33696614,32284046,28516545,24743946,18628700,13375121,9204938,6843607,527968,63.8232,1.0217,0.42,Yemen,ye,26853007
232,63,ZMB,Zambia,Lusaka,Africa,20017675,18927715,16248230,13792086,9891136,7686401,5720438,4281671,752612,26.5976,1.0280,0.25,Zambia,zm,15736004


In [23]:

world_data_df.loc[world_data_df['geo_country'] == 'France']

,Rank,CCA3,Country/Territory,Capital,Continent,2022 Population,2020 Population,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage,geo_country,iso_two,pop_change
68,23,FRA,France,Paris,Europe,64626628,64480053,63809769,62444567,58665453,56412897,53713830,50523586,551695,117.1419,1.0015,0.81,France,fr,14103042
145,185,NCL,New Caledonia,Nouméa,Oceania,289950,286403,283032,261426,221537,177264,148599,110982,18575,15.6097,1.0075,0.00,France,fr,178968


In [ ]:
base_url = "https://api.geoapify.com/v1/geocode/search"
params = {
    "apiKey":geoapify_key,
    "format":"json",
    "limit": 1
}

mapcheck = px.data.gapminder().query("year==2007")
mapcheck = mapcheck[['country', 'continent', 'iso_alpha']]
mapcheck = mapcheck.reindex(columns = mapcheck.columns.tolist() + ['geo_country'])
for index, row in mapcheck.iterrows():
    params['country'] = mapcheck.loc[index, 'country']
    current_country = requests.get(base_url, params).json()
    
    try: 
        mapcheck.loc[index, 'geo_country'] = current_country['results'][0]['country']
        print(f"Indexing: {mapcheck.loc[index, 'geo_country']}")
    except IndexError:
        print(current_country)
        mapcheck = mapcheck.drop(labels=index, axis=0)
        


In [ ]:
mapcheck['geo_country'].value_counts()


In [ ]:
mapcheck.loc[mapcheck['geo_country'] == 'Democratic Republic of Congo']

In [ ]:
mapcheck.loc[mapcheck['geo_country'] == 'South Korea']

In [ ]:
mapcheck.loc[mapcheck['geo_country'] == 'China']

In [ ]:
world_data_df = world_data_df.rename(columns={'Country/Territory': 'country'})


world_data = pd.merge(world_data_df, mapcheck, on='geo_country', how='inner')
# world_data['country_y'].value_counts()
# for index, row in world_data.iterrows():
#     if world_data.loc[index, 'country_x'] != world_data.loc[index, 'country_y']:
#         print(world_data.loc[index, 'country_x'])
        


# duplicates = world_data.loc[world_data.duplicated(subset=['country_x', 'country_y']), 'country_x'].unique()

# duplicate_df = []
# for x in range(len(duplicates)):
#     countryid = duplicates[x]
#     for y in range(len(world_data)):
#         if countryid == world_data.loc[y, 'country_x']:
#             duplicate_df.append(world_data.loc[y, :])
        
# duplicate_all = pd.DataFrame(duplicate_df)
# duplicate_all

# for x in range(len(duplicates)):
#     countryid = duplicates[x]
#     for y in range(len(world_data)):
#         if countryid == world_data.loc[y, 'country_x']:
#             world_data.drop([y], inplace=True)

# duplicates = world_data.loc[world_data.duplicated(subset=['country_x', 'country_y']), 'country_x'].unique()

In [ ]:
for index, row in world_data_df.iterrows():
    if world_data_df.loc[index, 'Country/Territory'] != world_data_df.loc[index, 'geo_country']:
        print(world_data_df.loc[index, 'Country/Territory'])

In [ ]:
world_data_list = []
for index, row in world_data_df.iterrows():
    world_data_list.append([world_data_df.loc[index, '1970 Population'], world_data_df.loc[index, '1980 Population'], world_data_df.loc[index, '1990 Population'],
                            world_data_df.loc[index, '2000 Population'], world_data_df.loc[index, '2010 Population'], world_data_df.loc[index, '2015 Population'],
                            world_data_df.loc[index, '2020 Population'], world_data_df.loc[index, '2022 Population']])

world_data_list
world_data_graph = pd.DataFrame({'country': world_data_df['geo_country'],
                                'area': world_data_df['Area (km²)'],
                                 'continent': world_data_df['Continent'],
                                'population': world_data_list})



In [ ]:
# https://coderzcolumn.com/tutorials/data-science/how-to-convert-static-maps-geopandas-to-interactive-maps-hvplot
# https://towardsdatascience.com/how-to-visualize-data-on-top-of-a-map-in-python-using-the-geoviews-library-part-2-e61a48ee6c3d

# shapes_df = gpd.read_file('map_shapes/ne_50m_admin_0_countries.shp', driver='ESRI Shapefile')

In [7]:
import plotly.express as px

# df = px.data.gapminder().query("year==2007")
# df
fig = px.choropleth(world_data_df, locations="CCA3",
                    color="Growth Rate", 
                    hover_name="Country/Territory",
                    color_continuous_scale=px.colors.sequential.RdBu)
fig.show()